In [23]:
import torch
import cv2
import numpy as np
from PIL import Image
import torchvision.transforms as T

from utils.util import non_max_suppression, scale_coords

In [27]:
# 1) Load model
ckpt = torch.load("weights/best.pt", map_location="cpu")
model = ckpt["model"].float()    # keep float32 for safety
model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# 2) Load and preprocess image
WEIGHT_PATH = "weights/best.pt"
img_path = "dataset/Highway.png"
#VietNam_street
#Highway
img = Image.open(img_path).convert("RGB")
orig_w, orig_h = img.size

transform = T.Compose([
    T.Resize((640, 640)),
    T.ToTensor()
])
x = transform(img).unsqueeze(0).to(device)

# If model params are fp16, cast input too
if next(model.parameters()).dtype == torch.half:
    x = x.half()



C:\Users\TDV\AppData\Local\Temp\ipykernel_20460\2061751865.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load("weights/best.pt", map_location="cpu")


In [31]:
img.size

(2121, 1414)

In [32]:
# 3) Forward pass
with torch.no_grad():
    out = model(x)          # raw output
    if isinstance(out, (list, tuple)):
        out = out[0]
    # shape [B,84,8400] -> [B,8400,84]
    out = out.permute(0, 2, 1)

# 4) NMS (repo’s version)
detections = non_max_suppression(out, confidence_threshold=0.25, iou_threshold=0.45)[0]

# 5) Rescale boxes back to original image
in_h, in_w = 640, 640
out_h, out_w = orig_h, orig_w

if detections is not None and len(detections):  
    detections[:, :4] = scale_coords((in_h, in_w), detections[:, :4], (out_h, out_w))
    # 6) Draw results
    img_cv = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
    for *xyxy, conf, cls in detections:
        x1, y1, x2, y2 = map(int, xyxy)
        label = f"{int(cls)} {conf:.2f}"
        cv2.rectangle(img_cv, (x1, y1), (x2, y2), (0,255,0), 2)
        cv2.putText(img_cv, label, (x1, y1-2), cv2.FONT_HERSHEY_SIMPLEX,
                    0.6, (0,255,0), 2)

    cv2.imwrite("inference_result.jpg", img_cv)
    print("✅ Saved inference_result.jpg")
else:
    print("No detections found.")


✅ Saved inference_result.jpg
